In [ ]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

In [ ]:
df = pd.read_csv("tmdb_5000_credits.csv")

df = df[["movie_id", "title", "cast", "crew"]]

df = df.fillna("[]")

In [ ]:
def safe_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        return []

def get_top_cast(cast_str, n=3):
    cast_list = safe_eval(cast_str)
    return [c["name"] for c in cast_list[:n] if "name" in c]

def get_director(crew_str):
    crew_list = safe_eval(crew_str)
    for member in crew_list:
        if member.get("job") == "Director":
            return [member["name"]]
    return []

def make_soup(row):
    cast = get_top_cast(row["cast"])
    director = get_director(row["crew"])
    return " ".join(cast + director).lower()

In [ ]:
df["soup"] = df.apply(make_soup, axis=1)

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df["soup"])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

df["title_norm"] = df["title"].str.lower()
indices = pd.Series(df.index, index=df["title_norm"])
all_titles = df["title_norm"].tolist()


In [ ]:
def recommend_movie(title, top_n=5):
    title_norm = title.lower().strip()

    if title_norm not in indices:
        match = difflib.get_close_matches(title_norm, all_titles, n=1, cutoff=0.6)
        if not match:
            return f"Movie '{title}' not found in dataset."
        title_norm = match[0]

    idx = indices[title_norm]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top = sim_scores[1 : top_n + 1]

    recs = [df.iloc[i]["title"] for i, _ in top]
    return recs

movie_name = input("Enter a movie name: ")
recommendations = recommend_movie(movie_name, top_n=5)

print("\nTop 5 Recommended Movies for:", movie_name)
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

Enter a movie name: Avatar

Top 5 Recommended Movies for: Avatar
1. Aliens
2. Snow White: A Tale of Terror
3. Guardians of the Galaxy
4. Out of the Furnace
5. The Ice Storm
